In [1]:
import os
import ccxt
# import config
import schedule
import pandas as pd
pd.set_option('display.max_rows', None)
from datetime import datetime
import time
import pytz
import json
import numpy as np
# import warnings
# warnings.filterwarnings('ignore')
import sys
sys.path.insert(1, './strategy')
from importlib import reload  

In [8]:
odf = pd.read_csv ('order-data/orders.csv', index_col='id', dtype={'id': 'string', 'orderId': 'string', 'strategy': 'string'})
ondf = odf[odf.orderId.isnull()]
# pd.isna(odf['orderId'])
ondf

,date_entered,exchange,pair,strategy,order_params,orderId
id,,,,,,
10,2021-08-19 14:45:04.560470,kucoin,DOT/USDT,BuyLevelOrder,"{""trigger_price"": 25.26, ""qty"": 0.02}",<NA>
11,2021-08-19 14:45:05.560470,binance,BTC/USDT,BuyLevelOrder,"{""trigger_price"": 40000, ""qty"": 0.001}",<NA>
2,2021-08-19 14:45:06.560470,ftx,DOT/USDT,BuyLevelOrder,"{""trigger_price"": 25, ""qty"": 0.01}",<NA>


In [3]:
odf = pd.read_csv ('order-data/orders.csv', index_col='id', dtype={'id': 'string', 'orderId': 'string'})
ondf = odf[odf.status=='new']
for index, orow in ondf.iterrows():
    module = __import__(orow.strategy)
    reload(module)

In [18]:
exchanges = {}
odf = ondf = xnodf = config = False
def run_bot():
    global odf, ondf, xnodf
    global exchanges
    config = {}
    with open ('config.json') as js: 
        config = json.load(js)

    for exkey, exconf in config['exchanges'].items():
        if exconf['enabled'] and not exkey in exchanges:
            excred = exconf['cred']
            for crk, crv in excred.items():
                if crv[0] == "$" : 
                    excred[crk] = os.getenv(crv[1:])

            exchanges[exkey] = getattr(ccxt, exkey) (excred)
    
    print(f'{datetime.now()} Checking for new orders...')
    odf = pd.read_csv ('order-data/orders.csv', index_col='id', dtype={'id': 'string', 'order': 'string', 'succ_order': 'string', 'orderId': 'string'})
    ondf = odf[odf.orderId.isnull()]

    for exkey, exchange in exchanges.items():
        xnodf = ondf[ondf.exchange == exkey]
        symbols =  xnodf.symbol.unique()
        tickers = exchange.fetchTickers (symbols)

        for oind, orow in xnodf.iterrows():
            ticker = tickers[orow.symbol]
            # try:
            orobj = eval(orow.order, {"__builtins__": {}}, {'ticker': ticker})
            if orobj['trigger']:
                # pass
                print(f'{datetime.now()} Executing order {oind} ...')
                
                # print(orow.succ_order)
                # if not pd.isnull(orow.succ_order):
                #     print('unavailable')
                # continue
                order = exchange.create_order(orow.symbol, orobj['type'], orobj['side'], orobj['amount'], orobj['price'], orobj['params'])
                if order['id']:
                    if not pd.isnull(orow.succ_order):
                        print(f'    Executing successive order ...')
                        succorobj = eval(orow.succ_order, {"__builtins__": {}}, {'ticker': ticker, 'order': order})
                        succorder = exchange.create_order(orow.symbol, succorobj['type'], succorobj['side'], succorobj['amount'], succorobj['price'], succorobj['params'])
                        # print(succorder)
                    
                odf.loc[oind, "orderId"] = order['id']
                odf.to_csv('order-data/orders.csv', index=True)
            # except:
            #     print("***ERROR***", sys.exc_info())
            # else:
            #     pass

                

            # module = __import__(orow.strategy)
            # order_class = getattr(module, orow.strategy)
    #     order_params = json.loads(orow.order_params)
    #     order_instance = order_class(exchange, orow.pair, tickers[orow.pair], order_params)
            # order = order_instance.execute({})
    #     if order:
    #         odf.loc[oind, "orderId"] = order['id']
    #         odf.to_csv('order-data/orders.csv', index=True)


# schedule.every(5).seconds.do(run_bot)
# while True:
#     schedule.run_pending()
#     time.sleep(1)
# run_bot()


In [7]:
xnodf

,date_entered,exchange,symbol,strategy,order,succ_order,orderId
id,,,,,,,
10,2021-08-19 14:45:04.560470,kucoin,DOT/USDT,BuyLevelOrder,"{'trigger': ticker['ask'] <= 27, 'type': 'mark...","{'type': 'market', 'amount': order['amount'], ...",<NA>


In [38]:
orow = ondf.iloc[0]
odf.index[odf.id == orow.id][0]

0

AttributeError: 'Series' object has no attribute 'id'

In [8]:


# orecords
# odf

In [10]:
from pymongo import MongoClient
from pprint import pprint

client = MongoClient()
db=client.pkbot

odf = pd.read_csv ('order-data/orders.csv', index_col='id', dtype={'id': 'string', 'order': 'string', 'succ_order': 'string', 'orderId': 'string'})
orecords = json.loads(odf.T.to_json()).values()

db.orders.insert(orecords)


<ipython-input-10-879266fc0237>:10: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  db.orders.insert(orecords)


[ObjectId('612047a13aea6021b8841b1a'),
 ObjectId('612047a13aea6021b8841b1b'),
 ObjectId('612047a13aea6021b8841b1c'),
 ObjectId('612047a13aea6021b8841b1d'),
 ObjectId('612047a13aea6021b8841b1e')]

In [11]:
# df = read_mongo(db, 'myCollection')
data = pd.DataFrame(list(db.orders.find()))
data

,_id,date_entered,exchange,symbol,order,succ_order,orderId
0,612047a13aea6021b8841b1a,2021-08-19 14:45:04.560470,kucoin,DOT/USDT,"{'trigger': ticker['ask'] <= 29, 'type': 'mark...","{'type': 'market', 'amount': order['amount'], ...",611fd205a701f400065045fe
1,612047a13aea6021b8841b1b,2021-08-19 14:45:04.560470,binance,DOT/USDT,"{'trigger': ticker['ask'] <= 28, 'type': 'mark...",None,1758504533
2,612047a13aea6021b8841b1c,2021-08-19 14:45:04.560470,binance,DOT/USDT,"{'trigger': True, 'type': 'STOP_LOSS_LIMIT', '...",None,stop-limit
3,612047a13aea6021b8841b1d,2021-08-19 14:45:04.560470,binance,DOT/USDT,"{'trigger': ticker['ask'] <= 28, 'type': 'mark...","{'trigger': True, 'type': 'STOP_LOSS_LIMIT', '...",buy-and-stop-limit
4,612047a13aea6021b8841b1e,2021-08-19 14:45:04.560470,binance,DOT/USDT,"{'trigger': ticker['ask'] <= 28, 'type': 'mark...","{'trigger': True, 'type': 'STOP_LOSS_LIMIT', '...",1759893686


{'_id': ObjectId('6120536374e4a6e07ddd34ae'),
 'q': [{'margs': [], 'method': '', 'mparams': []}],
 'status': 'new'}


In [3]:
exchanges = {}
config = False
with open ('config.json') as js: 
        config = json.load(js)

for exkey, exconf in config['exchanges'].items():
        if exconf['enabled'] and not exkey in exchanges:
                excred = exconf['cred']
                for crk, crv in excred.items():
                        if crv[0] == "$" : 
                                excred[crk] = os.getenv(crv[1:])

                exchanges[exkey] = getattr(ccxt, exkey) (excred)

In [5]:
from pymongo import MongoClient
from bson.objectid import ObjectId
from pprint import pprint

client = MongoClient()
db=client.pkbot

tickers = {}

def run_bot():
  global exchanges, tickers
  
  print("Scanning for new orders ...")
  for csyms in db.xqueue.aggregate([    
      {'$match':{'status': "new"}},
      {'$unwind':"$q"},
      {'$group':{'_id': {'exchange':"$q.exchange", "symbol": {'$first' : "$q.margs"}}}},
      {'$group':{'_id': '$_id.exchange',"symbols": {'$addToSet': "$_id.symbol"}}},
      # {'$match':{'_id': qo['exchange']}}
  ]):
    exchange_name = csyms['_id']
    symbols = csyms['symbols']
    tickers[exchange_name] = exchanges[exchange_name].fetchTickers(symbols)
    # print(exchange_name, tickers[exchange_name])
  for qx in db.xqueue.find({'status' :'new'}):
    print(f"Processing q/{qx['_id']}")
    for idxqo, qo in enumerate(qx['q']):
      nqx = db.xqueue.find_one({'_id': qx.get('_id')})
      print(f'   qid/{qo["qid"]} status: {qo["status"]}')
      exchange_name = qo['exchange']
      symbol = qo['margs'][0]
      
      if qo['status'] != 'new':
        continue

      print(f'        Checking {exchange_name}: qid/{qo["qid"]} {qo["method"]} : {qo["margs"]}')
      
      exchange = exchanges[exchange_name]
      ticker = tickers[exchange_name][symbol]

      if not eval(qo['trigger'], {'ticker': ticker, 'qx' : nqx}):
        continue

      print(f'        Triggered {exchange_name}:  {qo["method"]} : {qo["margs"]}')
      res = getattr(exchange, qo['method'])(*qo['margs'])
      updres = db.xqueue.update_one({'_id' : qx.get('_id') }, {
        '$set': { 
          # f"q.{idxqo}.orderId": res['id'],
          f"q.{idxqo}.status": 'done',
          f"q.{idxqo}.res": res,
          }
      })

    if db.xqueue.find_one({'_id': qx.get('_id'), "q.status" : { "$ne": "done" }}) == None:
      db.xqueue.update_one({'_id' : qx.get('_id') }, {
          '$set': { 'status': 'done' }
        })


schedule.every(5).seconds.do(run_bot)
while True:
    schedule.run_pending()
    time.sleep(1)
# run_bot()

Scanning for new orders ...
Scanning for new orders ...
Scanning for new orders ...
Scanning for new orders ...
Scanning for new orders ...
Scanning for new orders ...
Scanning for new orders ...
Scanning for new orders ...
Scanning for new orders ...
Scanning for new orders ...
Scanning for new orders ...
Scanning for new orders ...
Scanning for new orders ...
Scanning for new orders ...
Scanning for new orders ...
Processing q/6121924f74e4a6e07ddd8e7f
   qid/1 status: new
        Checking kucoin: qid/1 create_market_buy_order : ['DOT/USDT', 0.2]
        Triggered kucoin:  create_market_buy_order : ['DOT/USDT', 0.2]
   qid/1 status: new
        Checking kucoin: qid/1 create_order : ['DOT/USDT', 'market', 'sell', '0.2', 50, {'stop': 'loss', 'stopPrice': 25}]
        Triggered kucoin:  create_order : ['DOT/USDT', 'market', 'sell', '0.2', 50, {'stop': 'loss', 'stopPrice': 25}]
Scanning for new orders ...
Scanning for new orders ...
Scanning for new orders ...
Scanning for new orders ...


KeyboardInterrupt: 

In [33]:
 r = db.xqueue.aggregate([    
      {'$match':{'_id': ObjectId("6121924f74e4a6e07ddd8e7f")}},
      {'$unwind':"$q"},
      # {'$group':{'_id': {'exchange':"$q.exchange", "symbol": {'$first' : "$q.margs"}}}},
      # {'$group':{'_id': '$_id.exchange',"symbols": {'$addToSet': "$_id.symbol"}}},
    #   {'$match':{'_id': qo['exchange']}}
      {'$group':{'_id': {'qid':"$q.qid", "status": "$q.status"}}},
])
list(r)


[{'_id': {'qid': 2, 'status': 'done'}}, {'_id': {'qid': 1, 'status': 'new'}}]

In [14]:
r = db.xqueue.aggregate([    
      {'$match':{'status': "new"}},
      {'$unwind':"$q"},
      {'$group':{'_id': {'exchange':"$q.exchange", "symbol": {'$first' : "$q.margs"}}}},
      {'$group':{'_id': '$_id.exchange',"symbols": {'$addToSet': "$_id.symbol"}}},
      # {'$match':{'_id': qo['exchange']}}
  ])
list(r)

[{'_id': 'kucoin', 'symbols': ['DOT/USDT']}]

In [37]:
r = db.xqueue.find_one({'_id': ObjectId("6121924f74e4a6e07ddd8e7f"), "q.status" : { "$ne": "done" }})
# pprint(r != None)
r

{'_id': ObjectId('6121924f74e4a6e07ddd8e7f'),
 'status': 'new',
 'q': [{'qid': 1,
   'status': 'new',
   'trigger': '1 == 1',
   'exchange': 'kucoin',
   'method': 'create_limit_buy_order',
   'margs': ['DOT/USDT', '1', '20'],
   'orderId': '6120854fa701f4000680476a'},
  {'qid': 2,
   'status': 'new',
   'trigger': '1 == 1',
   'exchange': 'kucoin',
   'method': 'create_limit_sell_order',
   'margs': ['DOT/USDT', '1', '40'],
   'orderId': '612199d6a701f40006ca0c2e'}]}

In [38]:

list(db.xqueue.find({'status' :'new'}))

[{'_id': ObjectId('6121924f74e4a6e07ddd8e7f'),
  'status': 'new',
  'q': [{'qid': 1,
    'status': 'new',
    'trigger': '1 == 1',
    'exchange': 'kucoin',
    'method': 'create_limit_buy_order',
    'margs': ['DOT/USDT', '1', '20'],
    'orderId': '6120854fa701f4000680476a'},
   {'qid': 2,
    'status': 'new',
    'trigger': '1 == 1',
    'exchange': 'kucoin',
    'method': 'create_limit_sell_order',
    'margs': ['DOT/USDT', '1', '40'],
    'orderId': '612199d6a701f40006ca0c2e'}]}]